In [ ]:
import warnings
from typing import Annotated, Literal, Sequence, TypedDict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langchain_community.document_loaders import WebBaseLoader
from langchain_community. vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq


In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os. environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

llm = ChatGroq(model_name="Gemma2-9b-It")

In [ ]:
llm.invoke("Hi")

In [ ]:
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"

In [ ]:
data = WebBaseLoader(url).load()

In [ ]:
data[0].metadata["description"]

In [ ]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-06-23-prompt-engineering/"
]

In [ ]:
docs = [WebBaseLoader(url).load() for url in urls]

In [ ]:
docs_list = [item for sublist in docs for item in sublist]

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100
    chunk_overlap=5
)

In [ ]:
doc_splits = splitter.split_documents(docs_list)

In [ ]:
vector_store = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings
)

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="retriever_blog_posts",
    description="Search and return information about Lilian Weng blog posts on LLM agent and prompt engineering"
)

In [ ]:
tools = [retriever_tool]

In [ ]:
retrieve = ToolNode(tools=tools)

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
def ai_assistant(state: AgentState):
    print("--- CALL AGENT ---")
    messages = state["messages"]
    print(f"This is my message: {messages}")

    if len(messages) > 1:
        last_message = messages[-1]
        question = last_message.content

        prompt = PromptTemplate(
        template="""

            You are a helpful assistant, whatever question has been asked to find out that in the given question and then answer.
            Here is the question: {question}
            
        """,
        input_variables=["question"]
        )
        chain = prompt | llm

        response = chain.invoke({"question": question})
        return {"messages": [response]}
    
    else:
        llm_with_tool = llm.bind_tools(tools=tools)
        response = llm_with_tool.invoke(messages)
        # response = handel_query(messages)
        return {"messages": [response]}

In [ ]:
class grade(BaseModel):
    binary_score: str = Field(description="Relevance score 'yes' or 'no'")

In [ ]:
def grade_documents(state: AgentState) -> Literal["Output_Generator", "Query_Rewritor"]:
    llm_with_structure_output = llm.with_structured_output(grade)

    prompt = PromptTemplate(
        template="""

            You are a grader deciding if a document is relevant to user's question.
            Here is the document: {context}
            Here is the user's question: {question}
            If the document talks about or contains information related to the user's question, mark it as relevant.
            Give a 'yes' or 'no' answer to show if the document is relevant to the question
        """,
        input_variables=["context", "question"]
    )

    chain = prompt | llm_with_structure_output

    messages = state["messages"]
    print(f"message from the grader: {messages}")
    last_message = messages[-1]
    question = messages[0].content
    docs = last_message.content
    scored_result = chain.invoke({"question": question, "context": docs})
    score = scored_result.binary_score

    if score == "Yes":
        print("--- DECISION: DOCS RELEVANT ---")
        return "generator" # This should be a node name
    
    else:
        print("--- DECISION: DOCS NOT RELEVANT ---")
        return "rewriter" # This should be a node name

In [ ]:
def generate(state: AgentState):
    print("--- GENERATE ---")
    messages = state["messages"]

    print(f"Here is message from generate: {messages}")

    question = messages[0].content
    last_message = messages[-1]
    docs = last_message.content

    prompt = hub.pull("rlm/rag-prompt")

    rag_chain = prompt | llm
    
    response = rag_chain.invoke({"context": docs, "question": question})
    print(f"This is my response: {response}")

    return {"messages": [response]}

In [ ]:
def rewrite(state:AgentState):
    print("--- TRANSFORM QUERY ---")
    messages = state["messages"]
    question = messages[0].content

    print(f"Here is message from rewrite: {messages}")

    message = [HumanMessage(content=f"""Look at the input and try to reason about the underlying semantic intent or meaning.
                            Here is the initial question: {question}
                            Formulate an improved question:""")]
    

    response = llm.invoke(message)
    return {"messages": [response]}

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("My_AI_Assistant", ai_assistant)
workflow.add_node("Vectore_Retriever", retrieve)
workflow.add_node("Output_Generator", generate)
workflow.add_node("Query_Rewriter", rewrite)

workflow.add_edge(START, "MY_AI_Assistant")

workflow.add_conditional_edges("MY_AI_Assistant", tools_condition, {"tools":"Vector_Retriever",END:END})

workflow.add_conditional_edges("Vector_Retriever",
                               grade_documents,
                               {"generator": "Output_Generator",
                                "rewritor": "Query_Rewritor"})
workflow.add_edge("Output_Generator", END)

In [ ]:
app = workflow.compile()

In [ ]:
try:
    display(Image(app.get_graph().draw_mermaid_png()))

except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
app.invoke({"messages": ["What is Autonomous Agent?"]})

In [ ]:
app.invoke({"messages": ["What is the Capital of USA?"]})